### Lunar Lander BNN_LV with PYMC3
Using ADVI sampling in pymc3 to sample from posterior of BNN_LV

In [1]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(1)

In [2]:
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp
import sys
import pymc3 as pm
import arviz as az
import seaborn as sns
import pickle
# from pymc3.theanof import MRG_RandomStreams, set_tt_rng
import theano.tensor as tt
from tqdm import tqdm
from numpy.random import default_rng
import time
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from utils.data_gen import sample_gaussian_mixture, generate_regression_outputs
from utils.BNN_pymc3 import BNN_LV as BNN_LV_pymc3
from utils.custom_callbacks_pymc3 import plot95ci, wb_scatter, build_wb_callback
from utils.decomposition import lunar_entropy_decompose
from utils.calculations import PPO

In [4]:
%config InlineBackend.figure_format = 'retina'

np.set_printoptions(precision=3, suppress=True)
RANDOM_SEED = 31418 #maybe make this same seed for all notebooks to get exact same data??
rng = default_rng(RANDOM_SEED)

## Lunar Lander

In [5]:
from utils.games import LunarLander2D

### Set up simulation and extract data

In [6]:
env = LunarLander2D(L=4, W=5, max_steps=20, seed=207)
no_action_policy = lambda state: (0,0)  # For any state, play the "do nothing" action.
random_policy = None  # If the policy is None, the simulator chooses an action at random.

# choose to run on the no-action policy
env.run(episodes=100, progress=100, policy=no_action_policy)

Episode 100/100 took 20 steps.


In [7]:
transitions = env.extract_transition_dataset()
transitions = transitions.sample(frac=1, replace=False, random_state=123)
transitions

,start_x,start_y,action_x,action_y,result_x,result_y
1031,4,3,0,0,4,2
320,3,0,0,0,3,0
1445,5,0,0,0,5,0
260,1,4,0,0,1,4
1308,5,0,0,0,5,0
...,...,...,...,...,...,...
1122,3,0,0,0,3,0
1346,2,0,0,0,2,0
1406,4,0,0,0,4,0
1389,5,0,0,0,5,0


In [8]:
# Build training data (ignore X dimension and try to use starting Y position and Y action to predict landing Y position):
X_train = transitions[['start_x','start_y','action_x','action_y']].to_numpy()
Y_train = transitions[['result_x','result_y']].to_numpy()

print('X :',X_train.shape)
print('Y :',Y_train.shape)


X : (1598, 4)
Y : (1598, 2)


### Setup BNN_LV architecture and perform MLE fit

In [9]:
# Parameters
gamma_lunar = 2 
sigma_lunar = 1

architecture_lunar = {'input_n':4, 
             'output_n':2, 
             'hidden_layers':[20,20],
             'biases' : [1,1,1],
             'activations' : ['relu', 'relu', 'linear'],
             'gamma':[gamma_lunar],
             'sigma':[sigma_lunar,sigma_lunar]}

bnn_lv_lunar = BNN_LV_pymc3(architecture=architecture_lunar, seed=1)

bnn_lv_lunar.fit(X_train, Y_train, step_size=0.01, max_iteration=5000, check_point=500, regularization_coef=None)

Iteration 0 lower bound 2646.096337433959; gradient mag: 6061.788603341931
Iteration 500 lower bound 0.22921482692358316; gradient mag: 0.701654750765489
Iteration 1000 lower bound 0.16745112375411556; gradient mag: 0.3678438077869452
Iteration 1500 lower bound 0.13787118880350424; gradient mag: 0.1974624817857982
Iteration 2000 lower bound 0.11687353213758936; gradient mag: 0.1297895781285146
Iteration 2500 lower bound 0.10243598328524456; gradient mag: 0.08147604806238404
Iteration 3000 lower bound 0.09403996953491084; gradient mag: 0.06523531391007817
Iteration 3500 lower bound 0.08969548468738456; gradient mag: 0.06261075664616812
Iteration 4000 lower bound 0.08731744799811035; gradient mag: 0.08022712256963357
Iteration 4500 lower bound 0.08465591981245585; gradient mag: 0.2514360889520257


### Prepare parameters for pymc3 sampling

In [10]:
p_mu = 0
p_sigma = 5
l_sigma = 0.25
lv_gamma = 2 

my_draws = 30000 


### Run pymc3

In [11]:
from fastprogress import fastprogress
fastprogress.printing = lambda: True

with pm.Model() as pm_model:

    # Prior on w (same shape as MLE)
    w_prior = pm.Uniform(name='w', lower=-0.01, upper=0.01, shape=bnn_lv_lunar.weights.shape) 

    # Latent variable prior (same shape as number of datapoints)
    lv_prior = pm.Uniform(name='z', lower=-0.01, upper=0.01, shape=(X_train.shape[0],1)) 

    y = pm.Normal('y', mu = bnn_lv_lunar.forward(X = X_train, input_noise = lv_prior, weights=w_prior),
                        sigma = l_sigma,
                        observed = Y_train)

    mean_field = pm.fit(method='advi',  
                        callbacks=[pm.callbacks.CheckParametersConvergence()], 
                        obj_optimizer=pm.adam(learning_rate=0.01), obj_n_mc=100)
    
    trace = mean_field.sample(draws=my_draws)
    
    #prior analysis
    prior_pc = pm.sample_prior_predictive()
    
    #posterior predictive
    ppc = pm.fast_sample_posterior_predictive(trace=trace)
    
    #generate inference data
    idata = az.from_pymc3(trace=trace, prior=prior_pc, posterior_predictive=ppc, log_likelihood=True)

Finished [100%]: Average Loss = 2.2669e+05


### Complete sampling and save/load samples obtained

In [12]:
np.save('./d4rl-atari/saved_samples/lunar_samples_pymc3ADVI-2.npy',trace['w'])


In [13]:
posterior_samples_lunar = np.load('./d4rl-atari/saved_samples/lunar_samples_pymc3ADVI-2.npy')


In [14]:
idata.to_netcdf('./idata_Ladvi_uni.nc')


'./idata_Ladvi_uni.nc'

In [15]:
idata = az.from_netcdf('./idata_Ladvi_uni.nc')

In [16]:
with open('./trace_Ladvi_uni.p', 'wb') as f:
    pickle.dump(trace, f)
    

In [17]:
with open('./trace_Ladvi_uni.p', 'rb') as f:
    trace = pickle.load(f)

In [18]:
with open('./model_Ladvi_uni.p', 'wb') as g:
    pickle.dump(pm_model, g)
    

In [19]:
with open('./model_Ladvi_uni.p', 'rb') as g:
    pm_model = pickle.load(g)

In [21]:
posterior_samples_lunar.shape

(30000, 1, 582)

In [22]:
#thinning by factor of 12
posterior_samples_lunar = posterior_samples_lunar[::12, :]
posterior_samples_lunar.shape

(2500, 1, 582)

In [ ]:
az.style.use("default")
N = 180 # number of 15 pt repeats
N2 = 30 # number of 15 pt repeats for calculating entropy
L = 40 # Number of y points to take per set of samples for epistemic uncertainty

# Run the entropy decomposition for the 2D chicken
lunar_decomp = lunar_entropy_decompose(bnn_lv_lunar, transitions, posterior_samples_pymc3_chicken, 
                                                N, N2, L, 'UQ_Ladvi_uni.png')

#### Negative log-likelihood calculation as quantitative performance measure

The lower, the better

In [ ]:
obs_logp = pm_model.y.logp
loglikelihood = np.array([obs_logp(p) for p in trace.points()])
nll = -1 * loglikelihood
min_nll = min(nll)
round(min_nll, 3)

### Bayesian/posterior non-identifiability analysis 

#### Plotting the PPO

In [ ]:
#plot prior and posterior in one plot for qualitative/visual overlap assessment

az.style.use("arviz-darkgrid")

fig, ax = plt.subplots(figsize=(12,8))
az.plot_ppc(idata, mean=False, observed=False, num_pp_samples=100, group='prior', colors=['green', 'green', 'green'], ax=ax)
az.plot_ppc(idata, mean=False, observed=False, num_pp_samples=100, group='posterior', ax=ax)
plt.xlim(-5, 10)
plt.savefig('PPO_Ladvi_uni.png', dpi=300)
plt.show()

#### Calculating the PPO (approximation)

In [ ]:
observed_data = idata.observed_data
priorpredictive_dataset = idata.prior_predictive 
postpredictive_dataset = idata.posterior_predictive

coords = {}

for key in coords.keys():
    coords[key] = np.where(np.in1d(observed_data[key], coords[key]))[0]
    
prior_xarray = priorpredictive_dataset.isel(coords)
post_xarray = postpredictive_dataset.isel(coords)
prior_array = prior_xarray['y'].stack(z=('chain', 'draw', 'y_dim_0', 'y_dim_1'))
post_array = post_xarray['y'].stack(z=('chain', 'draw', 'y_dim_0', 'y_dim_1'))

\>35% is weakly identifiable, indicating posterior non-identifiability. So, the posterior is identifiable if the PPO is below 35% (0.35)

In [ ]:
PPO(prior_array, post_array, 200)

### Data cloning analysis for assessing likelihood non-identifiability

Steps:  

1. Perform Bayesian inference with a likelihood based on the original data (K=1), using uninformative priors on all parameters. 

2. Record the posterior variance for the parameters.  

3. Create a data set consisting of K clones, that is, the data repeated K times.

4. Perform Bayesian inference with a likelihood based on the cloned data set of step 3, using uninformative priors on all parameters.

5. Record the posterior variance for the parameters. Scale this variance by dividing through by the posterior variance for K=1, found in step 2.

6. Repeat steps 3 to 5 for successively larger values of K.

7. If the scaled variance is approximately equal to 1/K for a parameter then that parameter is identifiable and can be estimated. If the standardized variance is much larger than 1/K then that parameter is non-identifiable and cannot be estimated. If there is at least on non-identifiable parameter, then the model with that set of data is parameter redundant. 


In [ ]:
#step 2
var_1 = idata.posterior.var(("chain", "draw"))

In [ ]:
#step 3
K = 10
clone = np.repeat(Y_train, K, axis=0)

In [ ]:
#step 3b
clone_X = np.repeat(X_train, K, axis=0)

In [ ]:
#step 4
from fastprogress import fastprogress
fastprogress.printing = lambda: True

with pm.Model() as pm_model2: 
    
    # Prior on w (same shape as MLE)
    w_prior2 = pm.Uniform(name='w', lower=-0.01, upper=0.01, shape=bnn_lv_chicken.weights.shape)
    
    # Latent variable prior (same shape as number of datapoints)
    lv_prior2 = pm.Uniform(name='z', lower=-0.01, upper=0.01, shape=(clone_X.shape[0],1))
    

    y2 = pm.Normal(name='y', mu = bnn_lv_chicken.forward(X = clone_X, input_noise = lv_prior2, weights=w_prior2),
                        sigma = l_sigma,
                        observed = clone)


    mean_field2 = pm.fit(method='advi',  
                        callbacks=[pm.callbacks.CheckParametersConvergence()], #[pm.callbacks.CheckParametersConvergence(diff='absolute')]
                        obj_optimizer=pm.adam(learning_rate=0.01), obj_n_mc=100)
    
    trace2 = mean_field2.sample(draws=my_draws)

    #prior analysis
    prior_pc2 = pm.sample_prior_predictive()
    
    #posterior predictive
    ppc2 = pm.fast_sample_posterior_predictive(trace=trace2)
    
    #generate inference data
    idata2 = az.from_pymc3(trace=trace2, prior=prior_pc2, posterior_predictive=ppc2, log_likelihood=True)

In [ ]:
#step 5
var_K = idata2.posterior.var(("chain", "draw"))

In [ ]:
#step 5b
scaled_var = var_K / var_1

In [ ]:
#step 7 check difference between scaled variance and 1/K for a parameter 
#if non-id, scaled_var >> 1/K

def difference(array, value):
    
    diff = []
    for i in array:
        delta = abs(i - value)
        diff.append(delta)
    
    return diff

def test_dif(diff, value, p):
    
    for i in diff:
        if i > (1+p)*value:
            return print("""Difference is significant, so there is non-identifiability.""")
        else:
            continue
    
    return print("""Difference is not significant, so the model is identifiable.""")

In [ ]:
#check difference between scaled variance and 1/K for a parameter 
#if non-id, scaled_var >> 1/K
scaled_var_z = scaled_var['z'][:,0].to_numpy()
scaled_var_w = scaled_var['w'][0,:].to_numpy()


test_dif(difference(scaled_var_z, 1/K), 1/K, 0.10)
test_dif(difference(scaled_var_w, 1/K), 1/K, 0.10)     

#### Assessment of mean for each chain for model (non-)identifiability
Calculate the mean for each chain. They should be all the same, otherwise info from one chain is transferred to another chain, indicating model non-identifiability. 

In [ ]:

mu = []
for i in idata.posterior.chain.values:
    mu.append(np.mean(idata.posterior['w'][i,:,0,1])) #for both w and z and result_x and result_y

print(mu)


## Plots and calculations not directly related to non-identifiability analysis but that might be of importance

In [ ]:
az.style.use("arviz-darkgrid")

In [ ]:
az.plot_trace(trace, var_names=['z'], compact=False);


In [ ]:
az.plot_trace(trace, var_names=['z'], compact=False);


In [ ]:
with pm_model:    
    axes = az.plot_pair(trace, var_names='w', scatter_kwargs={'alpha':0.5})

    for ax in axes.flat:
        ax.set_xlabel(None);
        ax.set_ylabel(None);

    axes[0,0].figure.tight_layout();

In [ ]:
with pm_model:
    pm.plot_posterior(trace)